In [43]:
%%bash
cd ~/darknet
./darknet detect cfg/yolov3.cfg yolov3.weights data/dog.jpg>result.txt

layer     filters    size              input                output
    0 conv     32  3 x 3 / 1   608 x 608 x   3   ->   608 x 608 x  32  0.639 BFLOPs
    1 conv     64  3 x 3 / 2   608 x 608 x  32   ->   304 x 304 x  64  3.407 BFLOPs
    2 conv     32  1 x 1 / 1   304 x 304 x  64   ->   304 x 304 x  32  0.379 BFLOPs
    3 conv     64  3 x 3 / 1   304 x 304 x  32   ->   304 x 304 x  64  3.407 BFLOPs
    4 res    1                 304 x 304 x  64   ->   304 x 304 x  64
    5 conv    128  3 x 3 / 2   304 x 304 x  64   ->   152 x 152 x 128  3.407 BFLOPs
    6 conv     64  1 x 1 / 1   152 x 152 x 128   ->   152 x 152 x  64  0.379 BFLOPs
    7 conv    128  3 x 3 / 1   152 x 152 x  64   ->   152 x 152 x 128  3.407 BFLOPs
    8 res    5                 152 x 152 x 128   ->   152 x 152 x 128
    9 conv     64  1 x 1 / 1   152 x 152 x 128   ->   152 x 152 x  64  0.379 BFLOPs
   10 conv    128  3 x 3 / 1   152 x 152 x  64   ->   152 x 152 x 128  3.407 BFLOPs
   11 res    8                 152 x 

In [1]:
import os
os.chdir("/home/soham/darknet")
data = open('result.txt','r').read()
print(data)

data/dog.jpg: Predicted in 32.176195 seconds.
dog: 100%
Bounding Box:- Center_x = 0.291906, Center_y = 0.656986, Height = 0.569734, Width = 0.232555
truck: 92%
Bounding Box:- Center_x = 0.756160, Center_y = 0.217218, Height = 0.151043, Width = 0.271336
bicycle: 99%
Bounding Box:- Center_x = 0.448431, Center_y = 0.496672, Height = 0.562251, Width = 0.637248



In [2]:
def implist(data):
    imp_list = []
    first_split = data.split()
    for ele in first_split:
        if(ele[0].isdigit()==True):
            if(ele[-1] == ','):
                ele = ele.replace(",","")
            if(ele[-1] == '%'):
                ele = ele.replace("%","")
                ele = float(ele)
                ele = ele/100
            ele = float(ele)
            imp_list.append(ele)
        else:
            if(ele[-1] == ':'):
                ele = ele.replace(":","")
                imp_list.append(ele)
    imp_list = imp_list[2:len(imp_list)]
    return imp_list

In [3]:
imp_list = implist(data)
print(imp_list)

['dog', 1.0, 0.291906, 0.656986, 0.569734, 0.232555, 'truck', 0.92, 0.75616, 0.217218, 0.151043, 0.271336, 'bicycle', 0.99, 0.448431, 0.496672, 0.562251, 0.637248]


In [4]:
def breaking_into_components(imp_list):
    new_implist = []
    i = 0
    while(i<len(imp_list)):
        temp = imp_list[i:i+6]
        i = i+6
        new_implist.append(temp)
    return new_implist

In [5]:
imp_list = breaking_into_components(imp_list)
print(imp_list)

[['dog', 1.0, 0.291906, 0.656986, 0.569734, 0.232555], ['truck', 0.92, 0.75616, 0.217218, 0.151043, 0.271336], ['bicycle', 0.99, 0.448431, 0.496672, 0.562251, 0.637248]]


In [44]:
from __future__ import print_function
import numpy as np
import pandas as pd
import tensorflow as tf
import IPython
import sys
import numpy as np
from keras.models import load_model, Model
from keras.layers import Dense, Activation, Dropout, Input, LSTM, Reshape, Lambda, RepeatVector, Flatten
from keras.initializers import glorot_uniform
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras import backend as K

In [41]:
#This function uses loops, try something better if possible
#Each grid is in the format [confidence,xpos,ypos,height,width,person, bicycle, car, motorcycle, airplane, bus, train, truck, boat, traffic light, fire hydrant, street sign, stop sign, parking meter, bench, bird, cat, dog, horse, sheep, cow, elephant, bear, zebra, giraffe, hat, backpack, umbrella, shoe, eye, glasses, handbag, tie, suitcase, frisbee, skis, snowboard, sports ball, kite, baseball bat, baseball glove, skateboard, surfboard, tennis racket, bottle, plate, wine glass, cup, fork, knife, spoon, bowl, banana, apple, sandwich, orange, broccoli, carrot, hot dog, pizza, donut, cake, chair, couch, potted plant, bed, mirror, dining table, window, desk, toilet, door, tv, laptop, mouse, remote, keyboard, cell phone, microwave, oven, toaster, sink, refrigerator, blender, book, clock, vase, scissors, teddy bear, hair drier, toothbrush, hair brush] 
def make_final3d(random_3d,imp_list):
    labels = ["person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck", "boat", "traffic light", "fire hydrant", "street sign", "stop sign", "parking meter", "bench", "bird", "cat", "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "hat", "backpack", "umbrella", "shoe", "eye", "glasses", "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat", "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "plate", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake", "chair", "couch", "potted plant", "bed", "mirror", "dining table", "window", "desk", "toilet", "door", "tv", "laptop", "mouse", "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink", "refrigerator", "blender", "book", "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush", "hair brush"] 
    random_3d[:,:,0] = 0
    for obj in imp_list:
        xcenter = obj[2]
        ycenter = obj[3]
        xgrid_num = int(xcenter/(1/19)) + 1
        xcenter_wrt_grid = xcenter/(1/19) - int(xcenter/(1/19))
        ygrid_num = int(ycenter/(1/19)) + 1
        ycenter_wrt_grid = ycenter/(1/19) - int(ycenter/(1/19))
        random_3d[xgrid_num,ygrid_num,0] = obj[1]
        random_3d[xgrid_num,ygrid_num,1] = xcenter_wrt_grid
        random_3d[xgrid_num,ygrid_num,2] = xcenter_wrt_grid
        random_3d[xgrid_num,ygrid_num,3] = obj[4]
        random_3d[xgrid_num,ygrid_num,4] = obj[5]
        random_3d[xgrid_num,ygrid_num,5:96] = 0
        thing_num = 0
        for thing in labels:
            if obj[0] == thing:
                random_3d[xgrid_num][ygrid_num][thing_num+5] = 1
                break
            thing_num = thing_num+1
    return random_3d

In [55]:
random_3d = np.random.randn(19,19,96)
random_3d = random_3d - random_3d.astype(int)
final_3d = make_final3d(random_3d,imp_list)
print(final_3d.shape)

(19, 19, 96)


In [48]:
final_flat = final_3d.flatten()
print(final_flat)

[ 0.          0.37714595  0.656395   ...  0.41900016  0.04171136
 -0.084869  ]


In [56]:
print(final_flat.shape)

(34656,)
